# Belgium Rail Network EDA
Exploring GTFS data for NMBS/SNCB rail network.

In [ ]:
# Import libraries
from gtfs_functions import Feed
import networkx as nx
import folium
import pandas as pd
import matplotlib.pyplot as plt  # Optional for static plots

: 